In [1]:
# modality interpolation
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import logging
import copy
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import kneighbors_graph

from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

In [5]:
import pickle
new_X = pickle.load(open("X_afm.pkl",'rb'))
new_Y = pickle.load(open("Y_afm.pkl",'rb'))
new_f = pickle.load(open("total_f.pkl",'rb'))

x_silicon = pickle.load(open("x_silicon_afm.pkl",'rb'))
last_one_procceed_silicon_value = pickle.load(open("x_silicon_last_once_afm.pkl",'rb'))
x_silicon_len = pickle.load(open("x_silicon_len_afm.pkl",'rb'))
silicon_time = pickle.load(open("silicon_time_afm.pkl",'rb'))
sili_feas = pickle.load(open("silicon_faeas_afm.pkl",'rb'))

x_med = pickle.load(open("x_med_afm.pkl",'rb'))
# last_one_procceed_silicon_value = pickle.load(open("x_silicon_last_once_afm.pkl",'rb'))
x_med_len = pickle.load(open("x_med_len_afm.pkl",'rb'))
med_time = pickle.load(open("med_time_afm.pkl",'rb'))
med_feas = pickle.load(open("med_feas_afm.pkl",'rb'))

In [6]:
jieba_procceed_dp = pickle.load(open("jieba_procceed_dp_afm.pkl",'rb'))
dp_time = pickle.load(open("dp_time_afm.pkl",'rb'))

dp_mask = []
for i in jieba_procceed_dp:
    if len(i)== 1 and (i[0][0] == '无'):
        dp_mask.append(0)
    else:
        dp_mask.append(1)
print(sum(dp_mask))

cls_procceed_dp = []
for i in range(len(jieba_procceed_dp)):
    cur_cls_dp = []
    for j in range(len(jieba_procceed_dp[i])):
        cur_cur_dp = ['[CLS]']
        cur_cur_dp.extend(jieba_procceed_dp[i][j])
        cur_cls_dp.append(cur_cur_dp)
    cls_procceed_dp.append(cur_cls_dp)

831


In [7]:
jieba_procceed_in = pickle.load(open("jieba_procceed_in_afm.pkl",'rb'))
in_time = pickle.load(open("in_time_afm.pkl",'rb'))

in_mask = []
for i in jieba_procceed_in:
    if len(i)== 1 and (i[0][0] == '无'):
        in_mask.append(0)
    else:
        in_mask.append(1)
print(sum(in_mask))

cls_procceed_in = []
for i in range(len(jieba_procceed_in)):
    cur_cls_dp = []
    for j in range(len(jieba_procceed_in[i])):
        cur_cur_dp = ['[CLS]']
        cur_cur_dp.extend(jieba_procceed_in[i][j])
        cur_cls_dp.append(cur_cur_dp)
    cls_procceed_in.append(cur_cls_dp)

746


In [9]:
jieba_procceed_out = pickle.load(open("jieba_procceed_out_afm.pkl",'rb'))
out_time = pickle.load(open("out_time_afm.pkl",'rb'))

out_mask = []
for i in jieba_procceed_out:
    if len(i)== 1 and (i[0][0] == '无'):
        out_mask.append(0)
    else:
        out_mask.append(1)
print(sum(out_mask))

cls_procceed_out = []
for i in range(len(jieba_procceed_out)):
    cur_cls_dp = []
    for j in range(len(jieba_procceed_out[i])):
        cur_cur_dp = ['[CLS]']
        cur_cur_dp.extend(jieba_procceed_out[i][j])
        cur_cls_dp.append(cur_cur_dp)
    cls_procceed_out.append(cur_cls_dp)

746


In [10]:
sili_mask = []
for i in x_silicon:
    if len(i)== 1 and (i[0] == np.zeros(28)).all():
        sili_mask.append(0)
    else:
        sili_mask.append(1)
print(sum(sili_mask))

med_mask = []
for i in x_med:
    if len(i)== 1 and (i[0] == np.zeros(853)).all():
        med_mask.append(0)
    else:
        med_mask.append(1)
print(sum(med_mask))

dp_mask = []
for i in jieba_procceed_dp:
    if len(i)== 1 and (i[0][0] == '无'):
        dp_mask.append(0)
    else:
        dp_mask.append(1)
print(sum(dp_mask))

in_mask = []
for i in jieba_procceed_in:
    if len(i)== 1 and (i[0][0] == '无'):
        in_mask.append(0)
    else:
        in_mask.append(1)
print(sum(in_mask))

out_mask = []
for i in jieba_procceed_out:
    if len(i)== 1 and (i[0][0] == '无'):
        out_mask.append(0)
    else:
        out_mask.append(1)
print(sum(out_mask))



798
704
831
746
746


In [11]:
np.mean([len(x_silicon[i]) for i in range(len(x_silicon))])

1.0925480769230769

In [12]:
device = torch.device("cuda:1" if torch.cuda.is_available() == True else 'cpu')
# device = torch.device('cuda')
print("available device: {}".format(device))

available device: cuda:1


In [13]:
def batch_iter(x, y, dp, in_hos, out_hos, med, med_lens, sili, sili_lens,  \
               dp_mask, in_mask, out_mask, med_mask, sili_mask,batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size)  # 向下取整
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size]  # fetch out all the induces

        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx], dp[idx], in_hos[idx], out_hos[idx], med[idx], med_lens[idx], \
                             sili[idx], sili_lens[idx], dp_mask[idx], in_mask[idx], out_mask[idx], \
                             med_mask[idx], sili_mask[idx]))

        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
        batch_dp = [e[2] for e in examples]
        batch_in_hos = [e[3] for e in examples]
        batch_out_hos = [e[4] for e in examples]
        batch_med = [e[5] for e in examples]
        batch_med_lens = [e[6] for e in examples]
        batch_sili = [e[7] for e in examples]
        batch_sili_lens = [e[8] for e in examples]
        batch_dp_mask = [e[9] for e in examples]
        batch_in_mask = [e[10] for e in examples]
        batch_out_mask = [e[11] for e in examples]
        batch_med_mask = [e[12] for e in examples]
        batch_sili_mask = [e[13] for e in examples]

        yield batch_x, batch_y, batch_dp, batch_in_hos, batch_out_hos, \
        batch_med, batch_med_lens, batch_sili, batch_sili_lens, \
        [batch_dp_mask, batch_in_mask, batch_out_mask, batch_med_mask, batch_sili_mask]

In [14]:
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))


    return np.array(sents_padded)

In [15]:
new_X.shape

(832, 120)

In [17]:
from collections import namedtuple
import sys
from typing import List, Tuple, Dict, Set, Union
import torch
import torch.nn as nn
import torch.nn.utils
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import math, copy, time
from model_embeddings import ModelEmbeddings
# Hypothesis = namedtuple('Hypothesis', ['value', 'score'])
from torch.autograd import Variable

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.selu = nn.SELU()
        self.elu = nn.ELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.selu(self.w_1(x))))

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)



class NMT_tran(nn.Module):
   

    def __init__(self, embed_size, hidden_size, vocab, dropout_rate=0.2):
        """ Init NMT Model.

        @param embed_size (int): Embedding size (dimensionality)
        @param hidden_size (int): Hidden Size (dimensionality)
        @param vocab (Vocab): Vocabulary object containing src and tgt languages
                              See vocab.py for documentation.
        @param dropout_rate (float): Dropout probability, for attention
        """
        super(NMT_tran, self).__init__()
        self.model_embeddings = ModelEmbeddings(hidden_size, vocab)
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        self.vocab = vocab


      
        c = copy.deepcopy
        attn = MultiHeadedAttention(8, self.hidden_size)
        ff = PositionwiseFeedForward(self.hidden_size, self.hidden_size*4, self.dropout_rate)
        self.position = PositionalEncoding(embed_size, dropout_rate)
        self.encoder = Encoder(EncoderLayer(hidden_size, c(attn), c(ff), dropout_rate), 1)

        self.high_encoder = Encoder(EncoderLayer(hidden_size, c(attn), c(ff), dropout_rate), 1)
       
        self.opt = nn.Linear(
            in_features=(hidden_size), out_features=1, bias=False)

        self.sigmoid = nn.Sigmoid()
        self.selu = nn.SELU()
        self.elu = nn.ELU()
      
        self.dropout = nn.Dropout(p=dropout_rate)
    

    def forward(self, source: List[List[List[str]]]) -> torch.Tensor:
        
        # Compute sentence lengths
        source_lengths = [len(s) for s in source]

        total_src = []
        for i in source:
            total_src.extend(i)

        # Convert list of lists into tensors
        total_src_padded = self.vocab.src.to_input_tensor(
            total_src, device=self.device)   # Tensor: (src_len, b)
        
        if total_src_padded.shape[0]>400:
            total_src_padded = total_src_padded[:400,:]
#         print(total_src_padded.shape)
        
        total_src_lengths = [len(s) for s in total_src]
        
       
        enc_hiddens, first_hidden = self.encode(
            total_src_padded)

        last_hiddens = first_hidden
#         print(last_hiddens)

        return2source = []
        return2source_counter = 0
        for i in source_lengths:
            return2source.append(last_hiddens[return2source_counter:return2source_counter+i])
            return2source_counter += i

        assert(len(total_src_lengths) == return2source_counter)
        
#         print(return2source[0])

        
        
        return2source_padded = self.pad_sentence_embeddings(return2source, torch.zeros_like(return2source[0][0],device=device))
#         print(return2source_padded[1])
        
        return2source_padded = torch.stack(return2source_padded)
#         print(return2source_padded.shape)
        return2source_padded_mask = (return2source_padded != torch.zeros_like(return2source[0][0],device=device))[:,:,0].unsqueeze(-2)
#         print(return2source_padded_mask.shape)
        
      
#         X_padded = pack_padded_sequence(return2source_padded.permute(1,0,2), source_lengths,enforce_sorted=False)
        
        enc_hiddens = self.high_encoder(return2source_padded, return2source_padded_mask)
#         print(enc_hiddens.shape)#b t h

#         opt = self.sigmoid(self.opt(enc_hiddens[:,0,:].squeeze(0)))
       
#         os = []
#         for i in range(len(source_lengths)):
#             os.append(enc_hiddens[i,source_lengths[i]-1].squeeze(0))

            
#         os = torch.stack(os)

        
        
        return enc_hiddens, source_lengths
#     enc_hiddens[:,0,:].squeeze(0)

    def pad_sentence_embeddings(self, sents, pad_token):
        """ Pad list of sentences according to the longest sentence in the batch.
        @param sents (list[list[str]]): list of sentences, where each sentence
                                        is represented as a list of words
        @param pad_token (str): padding token
        @returns sents_padded (list[list[str]]): list of sentences where sentences shorter
            than the max length sentence are padded out with the pad_token, such that
            each sentences in the batch now has equal length.
        """
        sents_padded = []
        mask = []

        # YOUR CODE HERE (~6 Lines)
        max_sent_count = max([len(sent) for sent in sents])
        for sent in sents:
            if len(sent) < max_sent_count:
                leftovers = [pad_token] * (max_sent_count - len(sent))
                sent = torch.cat((sent,torch.stack(leftovers)),dim=0)
                

                
            sents_padded.append(sent)
#             mask.append(cur_mask)
        # END YOUR CODE

        return sents_padded

    
    def encode(self, source_padded: torch.Tensor) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """ Apply the encoder to source sentences to obtain encoder hidden states.
            Additionally, take the final states of the encoder and project them to obtain initial states for decoder.

        @param source_padded (Tensor): Tensor of padded source sentences with shape (src_len, b), where
                                        b = batch_size, src_len = maximum source sentence length. Note that
                                       these have already been sorted in order of longest to shortest sentence.
        @param source_lengths (List[int]): List of actual lengths for each of the source sentences in the batch
        @returns enc_hiddens (Tensor): Tensor of hidden units with shape (b, src_len, h*2), where
                                        b = batch size, src_len = maximum source sentence length, h = hidden size.
        @returns dec_init_state (tuple(Tensor, Tensor)): Tuple of tensors representing the decoder's initial
                                                hidden state and cell.
        """
        enc_hiddens, dec_init_state = None, None

        # print(source_padded.shape)
        source_padded = source_padded.permute(1,0) # b t 
#         print(source_padded.shape)
        src_mask = (source_padded != 0).unsqueeze(-2)
#         print(src_mask.shape)# b 1 t 
#         X = self.position(self.model_embeddings.source(source_padded))  # (src_len, b, e) new: b t e
        X = self.model_embeddings.source(source_padded)
#         print(X.shape)
       
        # X = pack_padded_sequence(X, source_lengths)
        
        # enc_hiddens, (last_hidden, last_cell) = self.encoder(X)
        
        enc_hiddens = self.encoder(X, src_mask) # b t h
        first_hidden = enc_hiddens[:,0,:]

       
        return enc_hiddens, first_hidden



    @property
    def device(self) -> torch.device:
        """ Determine which device to place the Tensors upon, CPU or GPU.
        """
        return self.model_embeddings.source.weight.device


In [18]:
from docopt import docopt
import numpy as np
from typing import List, Tuple, Dict, Set, Union
from tqdm import tqdm
from utils1 import read_corpus
from vocab import Vocab, VocabEntry

vocab = Vocab.build(cls_procceed_dp+cls_procceed_in+cls_procceed_out, 50000, 2)

initialize source vocabulary ..
number of word types: 10672, number of word types w/ frequency >= 2: 7969


In [20]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# def tile(a, dim, n_tile):
#     init_dim = a.size(dim)
#     repeat_idx = [1] * a.dim()
#     repeat_idx[dim] = n_tile
#     a = a.repeat(*(repeat_idx))
#     order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
#     return torch.index_select(a, dim, order_index)

class PositionwiseFeedForward(nn.Module): # new added
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.selu = nn.SELU()
        self.elu = nn.ELU()

    def forward(self, x):
        return self.w_2(self.dropout(self.selu(self.w_1(x))))

# class PositionwiseFeedForwardConv(nn.Module):

#     def __init__(self, model_dim=512, ffn_dim=2048, dropout=0.0):
#         super(PositionalWiseFeedForward, self).__init__()
#         self.w1 = nn.Conv1d(model_dim, ffn_dim, 1)
#         self.w2 = nn.Conv1d(model_dim, ffn_dim, 1)
#         self.dropout = nn.Dropout(dropout)
#         self.layer_norm = nn.LayerNorm(model_dim)

#     def forward(self, x):
#         output = x.transpose(1, 2)
#         output = self.w2(F.relu(self.w1(output)))
#         output = self.dropout(output.transpose(1, 2))

#         # add residual and norm layer
#         output = self.layer_norm(x + output)
#         return output

class PositionalEncoding(nn.Module): # new added
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=400):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
       
        
    def forward(self, x):
#         print(self.pe.shape)
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0 # 下三角矩阵

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)# b h t d_k
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k) # b h t t
    
    
#     if print_flagflag:
#         print("scores :")
#         print(scores[0][0])
    if mask is not None:# 1 1 t t
        scores = scores.masked_fill(mask == 0, -1e9)# b h t t 下三角
    p_attn = F.softmax(scores, dim = -1)# b h t t
    if dropout is not None:
        p_attn = dropout(p_attn)
#     if print_flag:
#         print("p_attn :")
#         print(p_attn[0][0])
    return torch.matmul(p_attn, value), p_attn # b h t v (d_k) 
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, self.d_k * self.h), 3)
        self.final_linear = nn.Linear(d_model, d_model)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1) # 1 1 t t

        nbatches = query.size(0)# b
        
        # d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))] # b h t d_k
        
       
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)# b h t v (d_k) 
        
#         print()
        
      
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)# b t d_model

        return self.final_linear(x)

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-7):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class vanilla_transformer_encoder(nn.Module):
    def __init__(self, input_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(vanilla_transformer_encoder, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        #self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        #self.d_k = d_k  
        #self.d_v = d_v # the two can be equal
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.embed = nn.Linear(self.input_dim, self.d_model)

        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 5000)
       
        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)
        self.output = nn.Linear(self.d_model, self.output_dim)
        
        self.output = nn.Linear(128, self.output_dim)
        
#         self.reduce_dim = nn.Linear(self.input_dim * 48, 128)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

        #self.sparsemax = Sparsemax(dim=0)

    def forward(self, input, mask):
        # input shape [batch_size, timestep, feature_dim]
#         demographic = demo_input
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)
        assert(self.d_model % self.MHD_num_head == 0)

     
        embed_input = self.embed(input)
#         print(input.shape)
        posi_input = self.PositionalEncoding(embed_input)# b t d_model
#         posi_input = embed_input# b t d_model

        
#         mask = subsequent_mask(time_step).to(device) # 1 t t 下三角
#         print(mask)
        contexts = self.SublayerConnection(posi_input, lambda x: self.MultiHeadedAttention(posi_input, posi_input, posi_input, mask))# b t d_model
        #contexts = self.MultiHeadedAttention(qs, ks, vs, mask)# b t h

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))# b t d_model
        
#         contexts = contexts.view(batch_size, 16 * time_step)

#
#         os = []
#         for j in range(contexts.shape[0]):
#             os.append(contexts[j,lens[j]-1])
            
#         os = torch.stack(os)
    
#         output = self.output(os)# b t 1
#         output = self.sigmoid(output)
          
        return contexts
    #, self.MultiHeadedAttention.attn

In [22]:
class MM_transformer_encoder(nn.Module):
    def __init__(self, input_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(MM_transformer_encoder, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        #self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        #self.d_k = d_k  
        #self.d_v = d_v # the two can be equal
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.embed = nn.Linear(self.input_dim, self.d_model)

        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 5000)
       
        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)
        self.output = nn.Linear(self.d_model, self.output_dim)
        
#         self.output = nn.Linear(128, self.output_dim)
        
#         self.reduce_dim = nn.Linear(self.input_dim * 48, 128)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

        #self.sparsemax = Sparsemax(dim=0)

    def forward(self, input, mask):
        # input shape [batch_size, timestep, feature_dim]
#         demographic = demo_input
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)
        assert(self.d_model % self.MHD_num_head == 0)

     
        input = self.embed(input)
#         print(input.shape)
        input = self.PositionalEncoding(input)# b t d_model
#         posi_input = embed_input# b t d_model

        
#         mask = subsequent_mask(time_step).to(device) # 1 t t 下三角
#         print(mask)
        contexts = self.SublayerConnection(input, lambda x: self.MultiHeadedAttention(input, input, input, mask))# b t d_model
        #contexts = self.MultiHeadedAttention(qs, ks, vs, mask)# b t h

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))# b t d_model
        
#         contexts = contexts.view(batch_size, 16 * time_step)

#
#         os = []
#         for j in range(contexts.shape[0]):
#             os.append(contexts[j,lens[j]-1])
            
#         os = torch.stack(os)
    
#         output = self.output(os)# b t 1
#         output = self.sigmoid(output)
          
        return contexts # b t h
    #, self.MultiHeadedAttention.attn

In [23]:
def length_to_mask(length, max_len=None, dtype=None):
    """length: B.
    return B x max_len.
    If max_len is None, then max of length will be used.
    """
    assert len(length.shape) == 1, 'Length shape should be 1 dimensional.'
    max_len = max_len or length.max().item()
    mask = torch.arange(max_len, device=length.device,
                        dtype=length.dtype).expand(len(length), max_len) < length.unsqueeze(1)
    if dtype is not None:
        mask = torch.as_tensor(mask, dtype=dtype, device=length.device)
    return mask

def init_weights(module):
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.normal_(mean=0.0, std=0.02)
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)

    if isinstance(module, nn.Linear) and module.bias is not None:
        module.bias.data.zero_()

In [24]:
def euclidean_dist(x, y):
    b = x.size(0)
    xx = torch.pow(x, 2).sum(1, keepdim=True).expand(b, b)
    yy = torch.pow(y, 2).sum(1, keepdim=True).expand(b, b).t()
    dist = xx+yy-2*torch.mm(x, y.t())
    return dist

def guassian_kernel(source, kernel_mul=2.0, kernel_num=1, fix_sigma=None):
    n = source.size(0)
    L2_distance = euclidean_dist(source, source)
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n**2-n)
        
#     print(bandwidth)
    bandwidth /= kernel_mul ** (kernel_num//2)
    bandwidth_list = [bandwidth*(kernel_mul**i) for i in range(kernel_num)]
    kernel_val = [torch.exp(-L2_distance/bandwidth_temp) for bandwidth_temp in bandwidth_list]
    return sum(kernel_val)/len(kernel_val)

In [25]:
# Our MAPLE framework
class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
#         print(self.dropout)
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha# 4 leakyrelu
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)
        self.attention = None

    def forward(self, input, adj):# V N, V V
        h = torch.mm(input, self.W)# V O
        N = h.size()[0]# NUM OF V

        # V*V O ->123412341234, V*V O -> 111222333444, V V 2O
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))# V V

        zero_vec = -9e15*torch.ones_like(e)# V V
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        self.attention = attention
        attention = F.dropout(attention, self.dropout, training=self.training)# V V
        h_prime = torch.matmul(attention, h)# V N

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime
        

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])
    
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(in_features, out_features).float())
        if bias:
            self.bias = Parameter(torch.Tensor(out_features).float())
        else:
            self.register_parameter('bias', None)
        self.initialize_parameters()

    def initialize_parameters(self):
        std = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-std, std)
        if self.bias is not None:
            self.bias.data.uniform_(-std, std)

    def forward(self, adj, x):
        y = torch.mm(x.float(), self.weight.float())
        output = torch.mm(adj.float(), y.float())
        if self.bias is not None:
            return output + self.bias.float()
        else:
            return output

In [28]:
class M3Care(nn.Module):
    def __init__(self, input_dim, hidden_dim, embed_size, output_dim=1, keep_prob=1):
        super(M3Care, self).__init__()

        # hyperparameters
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.keep_prob = keep_prob
        self.modal_num = 6
        
        self.NLP_model = NMT_tran(embed_size=embed_size,
                hidden_size=hidden_dim,
                dropout_rate= 1 - self.keep_prob,
                vocab=vocab).to(device)
        
        self.in_model = NMT_tran(embed_size=embed_size,
                hidden_size=hidden_dim,
                dropout_rate= 1 - self.keep_prob,
                vocab=vocab).to(device)
        
        self.out_model = NMT_tran(embed_size=embed_size,
                hidden_size=hidden_dim,
                dropout_rate= 1 - self.keep_prob,
                vocab=vocab).to(device)
        
        self.med_model = vanilla_transformer_encoder(input_dim = 853, d_model = self.hidden_dim, \
                                                     MHD_num_head = 4 , d_ff = self.hidden_dim*4, output_dim = 1).to(device)
        self.med2current = nn.Linear(self.hidden_dim, self.hidden_dim)
        
        self.sili_model = vanilla_transformer_encoder(input_dim = 28, d_model = self.hidden_dim,  \
                                                      MHD_num_head = 4 , d_ff = self.hidden_dim*4, output_dim = 1).to(device)
        self.sili2current = nn.Linear(self.hidden_dim, self.hidden_dim)
        
        self.linear1 = nn.Linear(self.input_dim, self.hidden_dim)
        self.linear_s = nn.Linear(28, self.hidden_dim)
#         self.linear2 = nn.Linear(self.hidden_dim *5, self.output_dim)
        self.MM_model = MM_transformer_encoder(input_dim = self.hidden_dim, d_model = self.hidden_dim,  \
                                                      MHD_num_head = 4 , d_ff = self.hidden_dim*4, output_dim = 1).to(device)
        self.MM_model2 = MM_transformer_encoder(input_dim = self.hidden_dim, d_model = self.hidden_dim,  \
                                                      MHD_num_head = 1 , d_ff = self.hidden_dim*4, output_dim = 1).to(device)


        self.token_type_embeddings = nn.Embedding(6, self.hidden_dim)
        self.token_type_embeddings.apply(init_weights)
        
        
        self.sep_token_embeddings = nn.Embedding(6, self.hidden_dim)
        self.sep_token_embeddings.apply(init_weights)
        
        self.PositionalEncoding = PositionalEncoding(self.hidden_dim, dropout = 0, max_len = 5000)
#         emb_data = self.token_type_embeddings.weight.data
#         self.token_type_embeddings = nn.Embedding(3, hs)
#         self.token_type_embeddings.apply(objectives.init_weights)
#         self.token_type_embeddings.weight.data[0, :] = emb_data[0, :]
#         self.token_type_embeddings.weight.data[1, :] = emb_data[1, :]
#         self.token_type_embeddings.weight.data[2, :] = emb_data[1, :]

        self.threshold = nn.Parameter(torch.zeros(size=(1,)))

        
        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax(-1)
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        self.selu = nn.SELU()
        self.elu = nn.ELU()
        
        
        self.proj1 = nn.Linear(self.hidden_dim *7, self.hidden_dim*2 )
        self.proj2 = nn.Linear(self.hidden_dim *5, self.hidden_dim *5)
        self.out_layer = nn.Linear(self.hidden_dim *2, self.output_dim)
        
        self.proj_in_out = nn.Linear(self.hidden_dim *2, self.hidden_dim)
        self.proj_med_sili = nn.Linear(self.hidden_dim *2, self.hidden_dim)
        
        self.simiProj = clones(torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
            self.selu,
            torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
            self.selu,
            torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
#             torch.nn.Softplus(),
#             torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
        ), self.modal_num)
        
        self.eps0 = nn.Parameter(torch.zeros(size=(1,))-0.3)
        self.eps1 = nn.Parameter(torch.zeros(size=(1,))-0.3)
        self.eps2 = nn.Parameter(torch.zeros(size=(1,))-0.3)
        self.eps3 = nn.Parameter(torch.zeros(size=(1,))-0.3)
        self.eps4 = nn.Parameter(torch.zeros(size=(1,))-0.3)
        self.eps5 = nn.Parameter(torch.zeros(size=(1,))-0.3)
        
        
       
        
    
    
        self.GCN = clones(GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True),self.modal_num)
        self.GCN_2 = clones(GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True),self.modal_num)
        self.GCN_3 = clones(GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True),self.modal_num)
        
        self.weight1 = clones(nn.Linear(self.hidden_dim, 1),self.modal_num)
        self.weight2 = clones(nn.Linear(self.hidden_dim, 1),self.modal_num)
        
        self.bn = nn.BatchNorm1d(self.hidden_dim)

    def forward(self, input,  pds, in_hos, out_hos, meds, med_lens, sili, sili_lens, d_i_o_m_s_masks):
        
        
        values_hidden00 = self.selu(self.linear1(input))# b 1

        
        pd_contexts, pd_lens = self.NLP_model(pds)# b t h
#         pd_contexts = self.selu(pd_contexts)
        dp_mask = length_to_mask(torch.from_numpy(np.array(pd_lens))).unsqueeze(1).to(device)
        
 
        med_mask = length_to_mask(med_lens).unsqueeze(1)
        med_contexts = self.med2current(self.selu(self.med_model(meds, med_mask)))# b 1
#         med_contexts = self.selu(self.med2current(self.selu(self.med_model(meds, med_mask))))# b 1
        
        
        in_contexts, in_lens = self.in_model(in_hos)# b 1
#         in_contexts = self.selu(in_contexts)
        

        
        out_contexts, out_lens = self.out_model(out_hos)# b 1
#         out_contexts = self.selu(out_contexts)

        
        sili_mask = length_to_mask(sili_lens).unsqueeze(1)
        sili_contexts = self.sili2current(self.selu(self.sili_model(sili, sili_mask)))# b 1
#         sili_contexts = self.selu(self.sili2current(self.selu(self.sili_model(sili, sili_mask))))# b 1
#         
#         print("sili_contexts0")
#         print(sili_contexts)
       
        
   
        nl_hidden00 = torch.zeros_like(pd_contexts[:, 0 ])
        in_hidden00 = torch.zeros_like(pd_contexts[:, 0 ])
        out_hidden00 = torch.zeros_like(pd_contexts[:, 0 ])
        med_hidden00 = torch.zeros_like(pd_contexts[:, 0 ])
        sili_hidden00 = torch.zeros_like(pd_contexts[:, 0 ])
        
#         for j in range(values_hidden00.shape[0]):
#             left_diag_hidden0 = left_diag_contexts[j, 0 ]
#             right_diag_hidden0 = right_diag_contexts[j, 0]
#             left_f_hidden0 = left_f[j]
#             right_f_hidden0 = right_f[j]
      
        for j in range(values_hidden00.shape[0]):
            nl_hidden00[j]=pd_contexts[j,pd_lens[j]-1]
            in_hidden00[j]=in_contexts[j,in_lens[j]-1]
            out_hidden00[j]=out_contexts[j,out_lens[j]-1]
            med_hidden00[j]=med_contexts[j,med_lens[j]-1]
            sili_hidden00[j]=sili_contexts[j,sili_lens[j]-1]
        

        
        dp_mask_ = torch.from_numpy(np.array(d_i_o_m_s_masks[0])).to(device).unsqueeze(1)# b 1
        in_mask_ = torch.from_numpy(np.array(d_i_o_m_s_masks[1])).to(device).unsqueeze(1)
        out_mask_ = torch.from_numpy(np.array(d_i_o_m_s_masks[2])).to(device).unsqueeze(1)
        med_mask_ = torch.from_numpy(np.array(d_i_o_m_s_masks[3])).to(device).unsqueeze(1)
        sili_mask_ = torch.from_numpy(np.array(d_i_o_m_s_masks[4])).to(device).unsqueeze(1)
        
        dp_mask2 = dp_mask_ * dp_mask_.permute(1,0)
        in_mask2 = in_mask_ * in_mask_.permute(1,0)
        out_mask2 = out_mask_ * out_mask_.permute(1,0)
        med_mask2 = med_mask_ * med_mask_.permute(1,0)
        sili_mask2 = sili_mask_ * sili_mask_.permute(1,0)
        
        
        
        nl_hidden_mat = guassian_kernel(self.bn(self.simiProj[0](nl_hidden00)), kernel_mul=2.0, kernel_num=3)
        nl_hidden_mat2 = guassian_kernel(self.bn(nl_hidden00), kernel_mul=2.0, kernel_num=3)
        nl_hidden_mat = (1-self.sigmoid(self.eps0))*nl_hidden_mat+self.sigmoid(self.eps0)*nl_hidden_mat2
        nl_hidden_mat = nl_hidden_mat*dp_mask2 
        
        in_hidden_mat = guassian_kernel(self.bn(self.simiProj[1](in_hidden00)), kernel_mul=2.0, kernel_num=3)
        in_hidden_mat2 = guassian_kernel(self.bn(in_hidden00), kernel_mul=2.0, kernel_num=3)
        in_hidden_mat = (1-self.sigmoid(self.eps1))*in_hidden_mat+self.sigmoid(self.eps1)*in_hidden_mat2
        in_hidden_mat = in_hidden_mat*in_mask2 
        
        
        out_hidden_mat = guassian_kernel(self.bn(self.simiProj[2](out_hidden00)), kernel_mul=2.0, kernel_num=3)
        out_hidden_mat2 = guassian_kernel(self.bn(out_hidden00), kernel_mul=2.0, kernel_num=3)
        out_hidden_mat = (1-self.sigmoid(self.eps2))*out_hidden_mat+self.sigmoid(self.eps2)*out_hidden_mat2
        out_hidden_mat = out_hidden_mat*out_mask2
        
        med_hidden_mat = guassian_kernel(self.bn(self.simiProj[3](med_hidden00)), kernel_mul=2.0, kernel_num=3)
        med_hidden_mat2 = guassian_kernel(self.bn(med_hidden00), kernel_mul=2.0, kernel_num=3)
        med_hidden_mat = (1-self.sigmoid(self.eps3))*med_hidden_mat+self.sigmoid(self.eps3)*med_hidden_mat2
        med_hidden_mat = med_hidden_mat*med_mask2
        
        sili_hidden_mat = guassian_kernel(self.bn(self.simiProj[4](sili_hidden00)), kernel_mul=2.0, kernel_num=3)
        sili_hidden_mat2 = guassian_kernel(self.bn(sili_hidden00), kernel_mul=2.0, kernel_num=3)
        sili_hidden_mat = (1-self.sigmoid(self.eps4))*sili_hidden_mat+self.sigmoid(self.eps4)*sili_hidden_mat2
        sili_hidden_mat = sili_hidden_mat*sili_mask2
        
        
        val_mat = guassian_kernel(self.bn(self.simiProj[5](values_hidden00)), kernel_mul=2.0, kernel_num=3)
        val_mat2 = guassian_kernel(self.bn(values_hidden00), kernel_mul=2.0, kernel_num=3)
        val_mat = (1-self.sigmoid(self.eps5))*val_mat+self.sigmoid(self.eps5)*val_mat2
#         val_mat = val_mat*dp_mask
        val_mask2 = torch.ones_like(sili_mask2)
    

        
        diff0 = torch.abs(torch.norm(self.simiProj[0](nl_hidden00)) - torch.norm(nl_hidden00))
        diff1 = torch.abs(torch.norm(self.simiProj[1](in_hidden00)) - torch.norm(in_hidden00))
        diff2 = torch.abs(torch.norm(self.simiProj[2](out_hidden00)) - torch.norm(out_hidden00))
        diff3 = torch.abs(torch.norm(self.simiProj[3](med_hidden00)) - torch.norm(med_hidden00))
        diff4 = torch.abs(torch.norm(self.simiProj[4](sili_hidden00)) - torch.norm(sili_hidden00))
        diff5 = torch.abs(torch.norm(self.simiProj[5](values_hidden00)) - torch.norm(values_hidden00))
        
        
      
        
        sum_of_diff = diff1+diff2+diff3+diff4+diff5
        
        similar_score = (val_mat+ nl_hidden_mat + in_hidden_mat + out_hidden_mat + med_hidden_mat + sili_hidden_mat)/ \
        (val_mask2+ dp_mask2 + in_mask2 + out_mask2 + med_mask2 + sili_mask2 )
#         similar_score = self.relu(similar_score - self.sigmoid(self.threshold)[0].detach())  
#         temp_thresh = self.sigmoid(self.threshold)[0]
#         bin_mask = similar_score>0
#         similar_score += bin_mask * temp_thresh
        similar_score = self.relu(similar_score - self.sigmoid(self.threshold)[0])  
        temp_thresh = self.sigmoid(self.threshold)[0]
        bin_mask = similar_score>0
        similar_score = similar_score + bin_mask * temp_thresh.detach()
        
#         similar_score = self.relu(similar_score - self.sigmoid(self.threshold))
        

        nl_hidden0 = self.selu(self.GCN[0](similar_score*dp_mask2, nl_hidden00))
        nl_hidden0 = self.selu(self.GCN_2[0](similar_score*dp_mask2, nl_hidden0))
#         nl_hidden0 = self.relu(self.GCN_3(similar_score*dp_mask2, nl_hidden0))

        in_hidden0 = self.selu(self.GCN[1](similar_score*in_mask2, in_hidden00))
        in_hidden0 = self.selu(self.GCN_2[1](similar_score*in_mask2, in_hidden0))  
#         in_hidden0 = self.relu(self.GCN_3(similar_score*in_mask2, in_hidden0))
        
#         right_diag_hidden0 = self.relu(self.GCN_3(similar_score2*right_diag_mask2, right_diag_hidden0)) 

        out_hidden0 = self.selu(self.GCN[2](similar_score*out_mask2, out_hidden00))
        out_hidden0 = self.selu(self.GCN_2[2](similar_score*out_mask2, out_hidden0))
#         out_hidden0 = self.relu(self.GCN_3(similar_score*out_mask2, out_hidden0))
        
        
        med_hidden0 = self.selu(self.GCN[3](similar_score*med_mask2, med_hidden00))
        med_hidden0 = self.selu(self.GCN_2[3](similar_score*med_mask2, med_hidden0))
        
        
        sili_hidden0 = self.selu(self.GCN[4](similar_score*sili_mask2, sili_hidden00))
        sili_hidden0 = self.selu(self.GCN_2[4](similar_score*sili_mask2, sili_hidden0))
#         sili_hidden0 = self.relu(self.GCN_3(similar_score*sili_mask2, sili_hidden0))
        
        
        
        dp_weight1 = torch.sigmoid(self.weight1[0](nl_hidden0))
        dp_weight2 = torch.sigmoid(self.weight2[0](nl_hidden00 ))
        dp_weight1 = dp_weight1/(dp_weight1+dp_weight2)
        dp_weight2 = 1-dp_weight1
        
        final_dp = dp_weight1*nl_hidden0+dp_weight2*nl_hidden00
        
        
        in_weight1 = torch.sigmoid(self.weight1[1](in_hidden0))
        in_weight2 = torch.sigmoid(self.weight2[1](in_hidden00 ))
        in_weight1 = in_weight1/(in_weight1+in_weight2)
        in_weight2 = 1-in_weight1
        
        final_in = in_weight1*in_hidden0+in_weight2*in_hidden00
        
        
        out_weight1 = torch.sigmoid(self.weight1[2](out_hidden0))
        out_weight2 = torch.sigmoid(self.weight2[2](out_hidden00 ))
        out_weight1 = out_weight1/(out_weight1+out_weight2)
        out_weight2 = 1-out_weight1
        
        final_out = out_weight1*out_hidden0+out_weight2*out_hidden00
        
        
        med_weight1 = torch.sigmoid(self.weight1[3](med_hidden0))
        med_weight2 = torch.sigmoid(self.weight2[3](med_hidden00 ))
        med_weight1 = med_weight1/(med_weight1+med_weight2)
        med_weight2 = 1-med_weight1
        
        final_med = med_weight1*med_hidden0+med_weight2*med_hidden00
        
        
        sili_weight1 = torch.sigmoid(self.weight1[4](sili_hidden0))
        sili_weight2 = torch.sigmoid(self.weight2[4](sili_hidden00 ))
        sili_weight1 = sili_weight1/(sili_weight1+sili_weight2)
        sili_weight2 = 1-sili_weight1
        
        final_sili = sili_weight1*sili_hidden0+sili_weight2*sili_hidden00
            
        
            
        for i in range(dp_mask_.shape[0]):
            if dp_mask_[i][0] != 1:
                pd_contexts[i,pd_lens[j]-1] = nl_hidden0[i]
            else:
                pd_contexts[i,pd_lens[j]-1] = final_dp[i]
                
        for i in range(dp_mask_.shape[0]):
            if in_mask_[i][0] != 1:
                in_contexts[i,in_lens[j]-1] = in_hidden0[i]
            else:
                in_contexts[i,in_lens[j]-1] = final_in[i] 
                
        for i in range(dp_mask_.shape[0]):
            if out_mask_[i][0] != 1:
                out_contexts[i,out_lens[j]-1] = out_hidden0[i]
            else:
                out_contexts[i,out_lens[j]-1] = final_out[i]
                
        for i in range(dp_mask_.shape[0]):
            if med_mask_[i][0] != 1:
                med_contexts[i,med_lens[i]-1] = med_hidden0[i]
            else:
                med_contexts[i,med_lens[i]-1] = final_med[i]
                
        for i in range(dp_mask_.shape[0]):
            if sili_mask_[i][0] != 1:
                sili_contexts[i,sili_lens[i]-1] = sili_hidden0[i]
            else:
                sili_contexts[i,sili_lens[i]-1] = final_sili[i]
                
        
        
        pd_contexts = self.PositionalEncoding(pd_contexts)  +  \
            self.token_type_embeddings(torch.zeros_like(dp_mask.permute(0,2,1).squeeze(-1)).to(device).long())

#         pd_contexts = torch.cat([self.sep_token_embeddings(torch.zeros_like(med_lens).to(device).long()).unsqueeze(1), pd_contexts], dim=1)
#         dp_mask = torch.cat([torch.ones_like(med_lens).to(device).unsqueeze(-1).unsqueeze(-1), dp_mask.int()], dim=-1)

        
        med_contexts = self.PositionalEncoding(med_contexts)+  \
            self.token_type_embeddings(torch.ones_like(med_mask.permute(0,2,1).squeeze(-1)).to(device).long())

#         med_contexts = torch.cat([self.sep_token_embeddings(torch.ones_like(med_lens).to(device).long()).unsqueeze(1), med_contexts], dim=1)
#         med_mask = torch.cat([torch.ones_like(med_lens).to(device).unsqueeze(-1).unsqueeze(-1), med_mask.int()], dim=-1)


        in_mask = length_to_mask(torch.from_numpy(np.array(in_lens))).unsqueeze(1).to(device)
        in_contexts = self.PositionalEncoding(in_contexts) +  \
            self.token_type_embeddings( 2*torch.ones_like(in_mask.permute(0,2,1).squeeze(-1)).to(device).long())

#         in_contexts = torch.cat([self.sep_token_embeddings(2*torch.ones_like(med_lens).to(device).long()).unsqueeze(1), in_contexts], dim=1)
#         in_mask = torch.cat([torch.ones_like(med_lens).to(device).unsqueeze(-1).unsqueeze(-1), in_mask.int()], dim=-1)

        
        out_mask = length_to_mask(torch.from_numpy(np.array(out_lens))).unsqueeze(1).to(device)
        out_contexts = self.PositionalEncoding(out_contexts) +  \
            self.token_type_embeddings(3*torch.ones_like(out_mask.permute(0,2,1).squeeze(-1)).to(device).long())

#         out_contexts = torch.cat([self.sep_token_embeddings(3*torch.ones_like(med_lens).to(device).long()).unsqueeze(1), out_contexts], dim=1)
#         out_mask = torch.cat([torch.ones_like(med_lens).to(device).unsqueeze(-1).unsqueeze(-1), out_mask.int()], dim=-1)

        
        sili_contexts = self.PositionalEncoding(sili_contexts) + \
            self.token_type_embeddings(4*torch.ones_like(sili_mask.permute(0,2,1).squeeze(-1)).to(device).long())

        z0 = torch.cat([pd_contexts, in_contexts, out_contexts, med_contexts, sili_contexts], dim=1)
        z0_mask = torch.cat([dp_mask, in_mask, out_mask, med_mask, sili_mask], dim=-1).int()
        
        z1 = self.selu(self.MM_model(z0, z0_mask))# b 1
        val_mask = length_to_mask(torch.ones((pd_contexts.shape[0],1)).int().squeeze()).unsqueeze(1).to(device).int()
#         print(z0_mask.shape)
#         print(val_mask.shape)

        z1 = torch.cat([values_hidden00.unsqueeze(1), z1], dim=1)
        z1_mask = torch.cat([val_mask, z0_mask], dim=-1)

        z2 = self.selu(self.MM_model2(z1, z1_mask))# b 1
    

        nl_hidden = []
        in_hidden = []
        out_hidden = []
        med_hidden = []
        sili_hidden = []
        for j in range(values_hidden00.shape[0]):
            nl_hidden.append(z2[j,1])
            in_hidden.append(z2[j,1 + pd_contexts.shape[1]])
            out_hidden.append(z2[j,1+pd_contexts.shape[1] + in_contexts.shape[1]])
            med_hidden.append(z2[j,1+pd_contexts.shape[1]  + in_contexts.shape[1] +\
                                 out_contexts.shape[1]  ])
            sili_hidden.append(z2[j,1+pd_contexts.shape[1]  +in_contexts.shape[1]+ \
                                 out_contexts.shape[1] +med_contexts.shape[1]])
            
        nl_hidden = torch.stack(nl_hidden)
        in_hidden = torch.stack(in_hidden)
        out_hidden = torch.stack(out_hidden)
        med_hidden = torch.stack(med_hidden)
        sili_hidden = torch.stack(sili_hidden)
        


        
        combined_hidden = torch.cat((values_hidden00, nl_hidden,in_hidden,out_hidden, \
                                     med_hidden, sili_hidden,z2[:,0,:]),-1)#b n h

        
        
        last_hs_proj = self.dropout(self.selu(self.proj1(combined_hidden)))

        
        output = self.sigmoid(self.out_layer(last_hs_proj))

        return output,sum_of_diff

In [29]:
def get_loss(y_pred, y_true, weight=None):
    loss = torch.nn.BCELoss(weight=weight)
    return loss(y_pred, y_true)

In [30]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn

epochs = 100
batch_size = 32

fold_count = 0
total_train_loss = []
total_valid_loss = []
global_best = 0
_global_best = 0


acc = []
auroc = []
auprc = []
minpse = []


_acc = []
_auroc = []
_auprc = []
_minpse = []

history = []
pad_token = np.zeros(853)
pad_token2 = np.zeros(28)
    # pad_token = np.zeros(17)
    # begin_time = time.time()
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

y_true_all = []
y_pred_all = []

_y_true_all = []
_y_pred_all = []
for train, test in kfold.split(new_X, new_Y):
    best_y_true = None
    best_y_pred = None
    
    _best_y_true = None
    _best_y_pred = None
        
    model = M3Care(input_dim = new_X.shape[-1], hidden_dim = 256, embed_size = 256, output_dim = 1, keep_prob=0.5).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = 1e-5)

    fold_count += 1
#     if fold_count !=3:
#         continue
# #     print(train)

    train_x = [new_X[i] for i in train ]
    train_x_pd = [cls_procceed_dp[i] for i in train ]
    train_y = [new_Y[i] for i in train ]
    train_x_med = [x_med[i] for i in train ]
    train_x_med_len = [x_med_len[i] for i in train ]
    
    train_x_sili = [x_silicon[i] for i in train ]
    train_x_sili_len = [x_silicon_len[i] for i in train ]
    
    train_x_in = [cls_procceed_in[i] for i in train ]
    train_x_out = [cls_procceed_out[i] for i in train ]
    
    train_x_dp_maks = [dp_mask[i] for i in train ]
    train_x_in_mask = [in_mask[i] for i in train ]
    train_x_out_mask = [out_mask[i] for i in train ]
    train_x_med_mask = [med_mask[i] for i in train ]
    train_x_sili_mask = [sili_mask[i] for i in train ]


#     train_x_len = [all_x_len[i] for i in train ]


    test_x = [new_X[i] for i in test ]
    test_x_pd = [cls_procceed_dp[i] for i in test ]

    test_y = [new_Y[i] for i in test ]
    test_x_med = [x_med[i] for i in test ]
    test_x_med_len = [x_med_len[i] for i in test ]
    
    test_x_sili = [x_silicon[i] for i in test ]
    test_x_sili_len = [x_silicon_len[i] for i in test ]
    
    test_x_in = [cls_procceed_in[i] for i in test ]
    test_x_out = [cls_procceed_out[i] for i in test ]
    
    test_x_dp_maks = [dp_mask[i] for i in test ]
    test_x_in_mask = [in_mask[i] for i in test ]
    test_x_out_mask = [out_mask[i] for i in test ]
    test_x_med_mask = [med_mask[i] for i in test ]
    test_x_sili_mask = [sili_mask[i] for i in test ]
#     test_x_len = [all_x_len[i] for i in test ]

    file_name = './model/fix-test-new-M30Care-200' 

    fold_train_loss = []
    fold_valid_loss = []
    best_acc = 0
    best_auroc = 0
    best_auprc = 0
    best_minpse = 0
    
    
    _best_acc = 0
    _best_auroc = 0
    _best_auprc = 0
    _best_minpse = 0

    for each_epoch in range(epochs):

        epoch_loss = []
        counter_batch = 0
        model.train()  

        for step, (batch_x, batch_y, batch_dp, batch_in_hos, batch_out_hos, 
                   batch_med, batch_med_lens, batch_sili, batch_sili_lens, d_i_o_m_s_masks) in enumerate(
                batch_iter(train_x, train_y, train_x_pd,train_x_in,train_x_out,
                           train_x_med, train_x_med_len, train_x_sili, train_x_sili_len, \
                           train_x_dp_maks, train_x_in_mask, train_x_out_mask, train_x_med_mask, \
                           train_x_sili_mask, batch_size, shuffle=True)):
            optimizer.zero_grad()

            batch_x = torch.tensor(batch_x, dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            
            batch_med = torch.tensor(pad_sents(batch_med, pad_token), dtype=torch.float32).to(device)
#             batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_med_lens = torch.tensor(batch_med_lens, dtype=torch.float32).to(device).int()
    
            batch_sili = torch.tensor(pad_sents(batch_sili, pad_token2), dtype=torch.float32).to(device)
#             batch_sili = torch.tensor(batch_sili, dtype=torch.float32).to(device)
#             batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_sili_lens = torch.tensor(batch_sili_lens, dtype=torch.float32).to(device).int()



            opt,sum_of_diff = model(batch_x, batch_dp, batch_in_hos, batch_out_hos, batch_med, batch_med_lens, \
                        batch_sili, batch_sili_lens, d_i_o_m_s_masks)
#             print(simi)

#             weight = torch.zeros_like(opt).to(device)
#             #print(opt.shape)
#             for i in range(opt.shape[0]):
#                 if  batch_y.unsqueeze(-1)[i,0] == torch.zeros(1).to(device):
#                     weight[i,0] = 1
#                 else:
#                     weight[i,0] = 3
           # print(weight)
#             print("batch_y.unsqueeze(-1)")
#             print(batch_y.unsqueeze(-1))
#             print("opt")
#             print(opt)
            BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             print("BCE_Loss")
#             print(BCE_Loss)
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)
#             print(BCE_Loss)
            model_loss =  BCE_Loss + sum_of_diff*1e-8
#     + 0.001*simi.sum()/batch_size
#             print(BCE_Loss)
#             print(simi.sum()/batch_size)

            loss = model_loss

            epoch_loss.append(BCE_Loss.cpu().detach().numpy())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
            optimizer.step()

#             if step % 50 == 0:
#                 print('Fold %d Epoch %d Batch %d: Train Loss = %.4f'%(fold_count,each_epoch, step, loss.cpu().detach().numpy()))

        epoch_loss = np.mean(epoch_loss)
        fold_train_loss.append(epoch_loss)

        #Validation
        y_true = []
        y_pred = []
        
        with torch.no_grad():
            model.eval()
            valid_loss = []
            valid_true = []
            valid_pred = []
#             print(test_x_pd)
            for batch_x, batch_y, batch_dp, batch_in_hos, batch_out_hos, batch_med, batch_med_lens, \
    batch_sili, batch_sili_lens, d_i_o_m_s_masks in  \
                batch_iter(test_x, test_y, test_x_pd, test_x_in,test_x_out, test_x_med, test_x_med_len, 
                   test_x_sili, test_x_sili_len, test_x_dp_maks, test_x_in_mask, test_x_out_mask, \
                           test_x_med_mask, test_x_sili_mask, batch_size):

                batch_x = torch.tensor(batch_x, dtype=torch.float32).to(device)
                batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)

                batch_med = torch.tensor(pad_sents(batch_med, pad_token), dtype=torch.float32).to(device)
    #             batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
                batch_med_lens = torch.tensor(batch_med_lens, dtype=torch.float32).to(device).int()
        
                batch_sili = torch.tensor(pad_sents(batch_sili, pad_token2), dtype=torch.float32).to(device)
#                 batch_sili = torch.tensor(batch_sili, dtype=torch.float32).to(device)
#             batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
                batch_sili_lens = torch.tensor(batch_sili_lens, dtype=torch.float32).to(device).int()


                optimizer.zero_grad()

                opt,_ = model(batch_x, batch_dp, batch_in_hos, batch_out_hos,batch_med, batch_med_lens, \
                            batch_sili, batch_sili_lens, d_i_o_m_s_masks)


                BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#                 print("BCE_Loss")
#                 print(BCE_Loss)
#                 REC_Loss = F.mse_loss(recon, batch_x, reduction='mean').to(device)

                valid_loss.append(BCE_Loss.cpu().detach().numpy())


                y_pred += list(opt.cpu().detach().numpy().flatten())
                y_true += list(batch_y.cpu().numpy().flatten())



            valid_loss = np.mean(valid_loss)
            fold_valid_loss.append(valid_loss)
#             y_pred += list(opt.cpu().detach().numpy().flatten())
#             y_true += list(batch_y.cpu().numpy().flatten())
#             y_true_all.append(y_true)
#             y_pred_all.append(y_pred)
            ret = metrics.print_metrics_binary(y_true, y_pred,verbose = 0)
            history.append(ret)
            #print()

#             if each_epoch % 10 == 0:
#                 print('Fold %d, epoch %d: Loss = %.4f Valid loss = %.4f acc = %.4f roc = %.4f'%(fold_count, each_epoch, fold_train_loss[-1], fold_valid_loss[-1], ret['acc'], ret['auroc']))
#                 metrics.print_metrics_binary(y_true, y_pred)

            cur_acc = ret['acc']
            if cur_acc > best_acc:
                best_y_true = y_true
                best_y_pred = y_pred
                
                best_acc = cur_acc
                best_auroc = ret['auroc']
                best_auprc = ret['auprc']
                best_minpse = ret['minpse']
                state = {
                    'net': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'epoch': each_epoch
                }
                torch.save(state, file_name+'_'+str(fold_count))

                if cur_acc > global_best:
                    global_best = cur_acc
                    state = {
                        'net': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'epoch': each_epoch
                    }
                    torch.save(state, file_name)
                    print('------------ Save best model - ACC: %.4f ------------'%cur_acc)
                    
            _cur_roc = ret['auroc']
            if _cur_roc > _best_auroc:
                _best_y_true = y_true
                _best_y_pred = y_pred
                
                _best_acc = ret['acc']
                _best_auroc = _cur_roc
                _best_auprc = ret['auprc']
                _best_minpse = ret['minpse']
                state = {
                    'net': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'epoch': each_epoch
                }
                torch.save(state, file_name+'_roc_'+str(fold_count))

                if _cur_roc > _global_best:
                    global_best = cur_acc
                    state = {
                        'net': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'epoch': each_epoch
                    }
                    torch.save(state, file_name)
                    print('------------ Save best model - ROC: %.4f ------------'%_cur_roc)

        print('Fold %d, Epoch %d, acc = %.4f, roc = %.4f, prc = %.4f'%(fold_count, each_epoch, ret['acc'], ret['auroc'], ret['auprc']))

    acc.append(best_acc)
    auroc.append(best_auroc)
    auprc.append(best_auprc)
    minpse.append(best_minpse)
    
    _acc.append(_best_acc)
    _auroc.append(_best_auroc)
    _auprc.append(_best_auprc)
    _minpse.append(_best_minpse)
    
    total_train_loss.append(fold_train_loss)
    total_valid_loss.append(fold_valid_loss)
    
    y_true_all.append(best_y_true)
    y_pred_all.append(best_y_pred)
    
    _y_true_all.append(_best_y_true)
    _y_pred_all.append(_best_y_pred)

print('acc %.4f(%.4f)'%(np.mean(acc), np.std(acc)))
print('auroc %.4f(%.4f)'%(np.mean(auroc), np.std(auroc)))
print('auprc %.4f(%.4f)'%(np.mean(auprc), np.std(auprc)))
print('minpse %.4f(%.4f)'%(np.mean(minpse), np.std(minpse)))  
    
print("==========================================================")

print('_acc %.4f(%.4f)'%(np.mean(_acc), np.std(_acc)))
print('_auroc %.4f(%.4f)'%(np.mean(_auroc), np.std(_auroc)))
print('_auprc %.4f(%.4f)'%(np.mean(_auprc), np.std(_auprc)))
print('_minpse %.4f(%.4f)'%(np.mean(_minpse), np.std(_minpse)))  
#     return np.mean(acc), np.std(acc), np.mean(auroc), np.std(auroc),np.mean(auprc), np.std(auprc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:127: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


------------ Save best model - ACC: 0.6071 ------------
------------ Save best model - ROC: 0.6070 ------------
Fold 1, Epoch 0, acc = 0.6071, roc = 0.6070, prc = 0.4517
Fold 1, Epoch 1, acc = 0.4286, roc = 0.5120, prc = 0.4404


/home/zch/EMR2/EMR/1Jiangsu/human_stat/utils/metrics.py:31: RuntimeWarning: invalid value encountered in double_scalars
  f1_score=2*prec1*rec1/(prec1+rec1)


Fold 1, Epoch 2, acc = 0.5714, roc = 0.4099, prc = 0.3416
Fold 1, Epoch 3, acc = 0.5238, roc = 0.3131, prc = 0.3118
Fold 1, Epoch 4, acc = 0.5714, roc = 0.3394, prc = 0.3431
------------ Save best model - ACC: 0.6190 ------------
Fold 1, Epoch 5, acc = 0.6190, roc = 0.4717, prc = 0.5166
Fold 1, Epoch 6, acc = 0.5714, roc = 0.3061, prc = 0.3985
Fold 1, Epoch 7, acc = 0.5000, roc = 0.3085, prc = 0.3945
Fold 1, Epoch 8, acc = 0.6190, roc = 0.4222, prc = 0.4339
------------ Save best model - ACC: 0.7143 ------------
------------ Save best model - ROC: 0.8566 ------------
Fold 1, Epoch 9, acc = 0.7143, roc = 0.8566, prc = 0.8411
------------ Save best model - ACC: 0.7262 ------------
Fold 1, Epoch 10, acc = 0.7262, roc = 0.7901, prc = 0.7609
Fold 1, Epoch 11, acc = 0.5714, roc = 0.5813, prc = 0.4453
------------ Save best model - ACC: 0.7619 ------------
Fold 1, Epoch 23, acc = 0.7619, roc = 0.8426, prc = 0.8104
------------ Save best model - ACC: 0.7857 ------------
------------ Save best 

/home/zch/EMR2/EMR/1Jiangsu/human_stat/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])


...

Fold 10, Epoch 61, acc = 0.5904, roc = 0.2005, prc = 0.3106


/home/zch/EMR2/EMR/1Jiangsu/human_stat/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])


Fold 10, Epoch 62, acc = 0.5904, roc = 0.1747, prc = 0.2932
Fold 10, Epoch 63, acc = 0.5663, roc = 0.1753, prc = 0.2734
Fold 10, Epoch 64, acc = 0.5783, roc = 0.1771, prc = 0.2725
Fold 10, Epoch 65, acc = 0.5663, roc = 0.2749, prc = 0.3064
Fold 10, Epoch 66, acc = 0.5301, roc = 0.4292, prc = 0.3973
Fold 10, Epoch 67, acc = 0.6265, roc = 0.4046, prc = 0.4034
Fold 10, Epoch 68, acc = 0.6265, roc = 0.3589, prc = 0.3707
Fold 10, Epoch 69, acc = 0.4940, roc = 0.3415, prc = 0.3784
Fold 10, Epoch 70, acc = 0.6145, roc = 0.4736, prc = 0.4455
------------ Save best model - ROC: 0.5750 ------------
Fold 10, Epoch 71, acc = 0.6145, roc = 0.5750, prc = 0.5007
Fold 10, Epoch 72, acc = 0.5060, roc = 0.4244, prc = 0.4053
Fold 10, Epoch 73, acc = 0.5301, roc = 0.2965, prc = 0.3016


/home/zch/EMR2/EMR/1Jiangsu/human_stat/utils/metrics.py:31: RuntimeWarning: invalid value encountered in double_scalars
  f1_score=2*prec1*rec1/(prec1+rec1)


Fold 10, Epoch 74, acc = 0.5783, roc = 0.3127, prc = 0.3200
Fold 10, Epoch 75, acc = 0.6386, roc = 0.4604, prc = 0.4380
Fold 10, Epoch 76, acc = 0.6386, roc = 0.3770, prc = 0.4229
Fold 10, Epoch 77, acc = 0.6386, roc = 0.4790, prc = 0.4672
Fold 10, Epoch 78, acc = 0.5542, roc = 0.4700, prc = 0.4605
Fold 10, Epoch 79, acc = 0.6024, roc = 0.4358, prc = 0.4453
Fold 10, Epoch 80, acc = 0.5663, roc = 0.3884, prc = 0.3610
Fold 10, Epoch 81, acc = 0.5663, roc = 0.5654, prc = 0.5729
------------ Save best model - ROC: 0.5792 ------------
Fold 10, Epoch 82, acc = 0.5904, roc = 0.5792, prc = 0.5600
------------ Save best model - ACC: 0.6747 ------------
------------ Save best model - ROC: 0.6549 ------------
Fold 10, Epoch 83, acc = 0.6747, roc = 0.6549, prc = 0.5956
Fold 10, Epoch 84, acc = 0.5663, roc = 0.5432, prc = 0.4942
Fold 10, Epoch 85, acc = 0.6265, roc = 0.5084, prc = 0.4788
Fold 10, Epoch 86, acc = 0.5904, roc = 0.5600, prc = 0.5187
Fold 10, Epoch 87, acc = 0.6265, roc = 0.5630, prc =

In [32]:
print('acc %.4f(%.4f)'%(np.mean(acc), np.std(acc)))
print('auroc %.4f(%.4f)'%(np.mean(auroc), np.std(auroc)))
print('auprc %.4f(%.4f)'%(np.mean(auprc), np.std(auprc)))
print('minpse %.4f(%.4f)'%(np.mean(minpse), np.std(minpse)))  
    
print("==========================================================")

print('_acc %.4f(%.4f)'%(np.mean(_acc), np.std(_acc)))
print('_auroc %.4f(%.4f)'%(np.mean(_auroc), np.std(_auroc)))
print('_auprc %.4f(%.4f)'%(np.mean(_auprc), np.std(_auprc)))
print('_minpse %.4f(%.4f)'%(np.mean(_minpse), np.std(_minpse))) 

acc 0.7740(0.0892)
auroc 0.8012(0.1072)
auprc 0.7637(0.1361)
minpse 0.7161(0.1170)
_acc 0.6706(0.1625)
_auroc 0.8521(0.0858)
_auprc 0.8354(0.1007)
_minpse 0.7504(0.1074)
